In [1]:
import pathlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
from orchid import (project_loader as opl)

In [4]:
import toolz.curried as toolz

In [5]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [6]:
project_filenames = {
    'permian-a': 'Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac',
    'gng': 'GnG_DemoProject_wDataFrames.ifrac',
}

In [7]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

{'permian-a': WindowsPath('c:/src/Orchid.IntegrationTestData/Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac'),
 'gng': WindowsPath('c:/src/Orchid.IntegrationTestData/GnG_DemoProject_wDataFrames.ifrac')}

In [8]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

{'permian-a': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x0000017B63FA79D0>,
 'gng': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x0000017B63FA7A30>}

In [9]:
def make_project_data_frames_by_id(project):
    return {df.ObjectId: df for df in project.DataFrames.Items}

project_net_data_frames_by_id = toolz.pipe(
    projects,
    toolz.valmap(make_project_data_frames_by_id),
)
project_net_data_frames_by_id

{'permian-a': {<System.Guid object at 0x0000017B63FA8490>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x0000017B63FA8460>,
  <System.Guid object at 0x0000017B63FA86D0>: <Orchid.FractureDiagnostics.Factories.DataFrames.ObservationSetDataFrame object at 0x0000017B63FA86A0>,
  <System.Guid object at 0x0000017B63FA88E0>: <Orchid.FractureDiagnostics.Factories.DataFrames.MicroseismicDataFrame object at 0x0000017B63FA88B0>},
 'gng': {<System.Guid object at 0x0000017B63FA8940>: <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x0000017B63FA8910>,
  <System.Guid object at 0x0000017B63FA8B80>: <Orchid.FractureDiagnostics.Factories.DataFrames.FaultTraceSetDataFrame object at 0x0000017B63FA8B50>,
  <System.Guid object at 0x0000017B63FA8DC0>: <Orchid.FractureDiagnostics.Factories.DataFrames.StageDataFrame object at 0x0000017B63FA8D90>,
  <System.Guid object at 0x0000017B63FAF040>: <Orchid.FractureDiagnostics.Factories.DataFrames.WellLogSet

In [12]:
def make_project_data_frames_by_name(project):
    return {df.Name: df for df in project.DataFrames.Items}

project_net_data_frames_by_name = toolz.pipe(
    projects,
    toolz.valmap(make_project_data_frames_by_name),
)
project_net_data_frames_by_name

{'permian-a': {'Project Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x0000017B63FAFA90>,
  'FDI Observations': <Orchid.FractureDiagnostics.Factories.DataFrames.ObservationSetDataFrame object at 0x0000017B63FAFB80>,
  'C3-Microseismic Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.MicroseismicDataFrame object at 0x0000017B63FAFBB0>},
 'gng': {'Project Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.ProjectDataFrame object at 0x0000017B63FAFBE0>,
  'Fault Trace Set Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.FaultTraceSetDataFrame object at 0x0000017B63FAFC10>,
  'Stage Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.StageDataFrame object at 0x0000017B63FAFC40>,
  'Well Log Set Data Frame 01': <Orchid.FractureDiagnostics.Factories.DataFrames.WellLogSetDataFrame object at 0x0000017B63FAFCA0>,
  'Horizon Marker Set Data Frame01': <Orchid.FractureDiagnostics.Factories.Dat

In [13]:
len(list(toolz.get('permian-a', project_net_data_frames_by_name)))

3